<a href="https://colab.research.google.com/github/marastika/fake-review-svm.catboost-dan-GAN/blob/main/GAN_Fake_Akun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LeakyReLU, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from tensorflow.keras.models import Sequential

In [ ]:
!pip install --upgrade nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('stopwords', download_dir='path_to_download_dir')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package stopwords to path_to_download_dir...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Load data with a different encoding
data = pd.read_csv("/content/Data scrapping komentar e-commerce.csv", encoding='ISO-8859-1')

In [ ]:
# Preprocessing function
def preprocess_text(text):
    # Lowercase
    text = text.lower()
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenization
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    # Join tokens
    text = ' '.join(tokens)
    return text

In [ ]:
# Preprocess the 'Review' column
data['Review /Komentar'] = data['Review /Komentar'].apply(preprocess_text)

In [ ]:
# Menampilkan nama-nama kolom yang ada dalam DataFrame
print(data.columns)

Index(['No', 'Nama pemilik', 'Nama Toko', 'Produk', 'Nama akun pembeli',
       'Profil gambar akun', 'Deteksi Akun', 'Review /Komentar',
       'Rating bintang', 'Sentimen', 'Source', 'Data_Review', 'Month', 'YEAR',
       'LINK', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18',
       'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22',
       'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26',
       'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30',
       'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34'],
      dtype='object')


In [ ]:
# Find the unique words in the preprocessed text
unique_words =set(' '.join(data['Review /Komentar']).split())

In [ ]:
# Define constants
latent_dim =100
input_dim =len(unique_words) # define the input dimension based on the preprocessing step

In [ ]:
# Define the Generator model
def build_generator(latent_dim):
    model = Sequential([
        Dense(128, input_dim=latent_dim, activation='relu'),
        Dense(latent_dim, activation='sigmoid')
    ])
    return model

In [ ]:
# Define the Discriminator model
def build_discriminator(input_dim):
    model = Sequential([
        Dense(256, input_dim=input_dim),
        LeakyReLU(alpha=0.2),
        Dropout(0.25),
        Dense(128),
        LeakyReLU(alpha=0.2),
        Dropout(0.25),
        Dense(1, activation='sigmoid')
    ])
    return model

In [ ]:
# Define the GAN model
def build_gan(generator, discriminator, latent_dim):
    discriminator.trainable = False

    gan_input = tf.keras.Input(shape=(latent_dim,))
    x = generator(gan_input)
    gan_output = discriminator(x)

    gan = Model(gan_input, gan_output)
    gan.compile(loss='binary_crossentropy', optimizer='adam')

    return gan

In [ ]:
# Set dimensions
latent_dim = 100  # Example latent dimension size
input_dim = 100   # Example input dimension size for the discriminator

# Initialize models
generator = build_generator(latent_dim)
discriminator = build_discriminator(input_dim)
gan = build_gan(generator, discriminator, latent_dim)

print(gan)


In [ ]:
# Generate new data using the trained generator
generated_data = generator.predict(np.random.randn(len(data), latent_dim))

42/42 [==============================] - 1s 21ms/step


In [ ]:
# Predict whether the generated data is fake or not using the discriminator model
predictions = discriminator.predict(generated_data)
print(predictions)

42/42 [==============================] - 0s 2ms/step
[[0.7069435 ]
 [0.6810395 ]
 [0.664957  ]
 ...
 [0.71145535]
 [0.6988979 ]
 [0.6920153 ]]


In [ ]:
# Calculate the average prediction score
average_score = np.mean(predictions)
print("Average Prediction Score:", average_score)

Average Prediction Score: 0.68153805


In [ ]:
# Convert the predictions to binary logic (1 for "not fake" and 0 for "fake")
binary_predictions = (predictions > average_score).astype(int)

In [ ]:
# Map binary values to corresponding labels
labels = ["fake" if pred == 0 else "not fake" for pred in binary_predictions]


In [ ]:
# Create ground truth labels
num_generated_samples = len(generated_data)
num_real_samples = len(data)
generated_labels = np.zeros(num_generated_samples)  # Label fake data as 0
real_labels = np.ones(num_real_samples)             # Label real data as 1

# Combine generated and real labels
ground_truth_labels = np.concatenate([generated_labels, real_labels])

In [ ]:
accuracy = accuracy_score(ground_truth_labels[:len(binary_predictions)], binary_predictions)
print("GAN Classifier Accuracy:", accuracy)

GAN Classifier Accuracy: 0.5026375282592314
